# Building gene families

Phylogenetic anlysis can only be performed on sequences that are evolutionarily related. In other words, the first thing we need to do is to organize all gene/protein sequences in our target genomes into families. 

There are many ways to do this, and different methods and thresholds will produce slighluty different results. 

For simplicity, here we are reproducing a fast and simple approach, based on a BLAST-like clustering using the programm MMSSeqs2. 



In [ ]:
%%bash 

set -x

BASE="all_prots"

# create mmseqs db
./mmseqs_linux createdb --dbtype 1 $BASE.faa $BASE.mmseqs

# run relaxed clustering
./mmseqs_linux cluster --threads 8 -c 0.1 --cov-mode 5 --cluster-mode 2  $BASE.mmseqs $BASE.clusters.mmseqs tmp/

# parse clustering results
./mmseqs_linux createtsv $BASE.mmseqs $BASE.mmseqs $BASE.clusters.mmseqs $BASE.clusters.tsv
cat  $BASE.clusters.tsv|datamash -g 1 countunique 2 collapse 2 > $BASE.clusters.folded.tsv

# Check how many clusters with at least 4 seqs?
awk '$2>3 ' $BASE.clusters.folded.tsv|wc -l



## Once we have inferred all gene families, we can generate a FASTA file for each of them. 

In [ ]:
# Lets create fasta files with at least 4 seqs
mkdir fasta/ -p 

Load all sequence into a dictionary. Whenever possible (i.e. you have enough memory), this is a very easy way to look up for sequences. 

Here we are using the `SeqGroup` object from ETE, but you could parse FASTA file with other tools such as the Biopython `Bio.SeqIO` 


In [3]:
import sys
from ete3 import SeqGroup

basename = "all_prots"

# Preload all prot and gene sequences into ETE3 SeqGroup instances. 
# (note that this may requires much memory if the number of proteoms is high. 
# In that case, you should switch to a iterative parsing statregy )
all_prots = SeqGroup("data/%s.faa" %basename)
all_genes = SeqGroup("data/%s.fna" %basename)

print("Loaded %d prot and %d gene sequences" %(len(all_prots), len(all_genes)))

Loaded 40385 prot and 41107 gene sequences


Let's now read the members of each gene family inferred, and create a FASTA file for them. 

Note that, for this project example, we will focus only on gene families with 3 o more members.


In [4]:
# Read the list of clusters produced and generate FASTA files for all families with 4 or more members.
cluster_tsv = "data/fam_clustering/all_prots.clusters.folded.tsv"
for line in open(cluster_tsv):
    cluname, size, raw_members = line.split('\t')
    if int(size) > 3: # Select only if 4 or more members (smaller families cannot)
        members = list(map(str.strip, raw_members.split(',')))
        with open('data/fasta/%s.faa' %cluname, "w") as FAA, open('data/fasta/%s.fna' %cluname, "w") as FNA:
            for m in members:
                print(">%s\n%s" %(m, all_prots.get_seq(m)), file=FAA)
                try:
                    print(">%s\n%s" %(m, all_genes.get_seq(m)), file=FNA)
                except KeyError:
                    print("Missing nt seq: ", m)
                    

Missing nt seq:  224324999.sul45501
Missing nt seq:  224324999.sul45504
Missing nt seq:  224324999.sul45505
Missing nt seq:  224324999.sul45503
Missing nt seq:  933801.Ahos_17051
Missing nt seq:  273063.STK_188951
Missing nt seq:  933801.Ahos_17053
Missing nt seq:  933801.Ahos_17054
Missing nt seq:  273063.STK_188953
Missing nt seq:  933801.Ahos_17052
Missing nt seq:  273063.STK_188952
